# Final Project: Challenge 9
Contributions From: Ryan Cottone

**Difficulty**: <span style="color:dark_red">$\bullet \bullet \bullet \bullet \bullet$</span>

Welcome to the final challenge! This one is quite tricky to figure out, but does not require large amounts of computing power or brute force.

**No modification has been made to the base code. Find the vulnerability that has been hidden the entire time.**

In [ ]:
%run client.ipynb
%run server.ipynb
from utils import *

In [ ]:
from base64 import b64encode, b64decode
from Crypto.Cipher import AES
from Crypto.Hash import HMAC, SHA256
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
import time

In [ ]:
ellipticCurve = [0, 7]
ellipticCurveModulus = 115792089237316195423570985008687907853269984665640564039457584007908834671663 
generatorPoint = (55066263022277343669578718895168534326250603453777594175500187360389116729240,
 32670510020758816978083085130507043184471273380659243275938904335757337482424)
generatorPointOrder =115792089237316195423570985008687907852837564279074904382605163141518161494337

You may or may not need the Interceptor class, but you can assume you have the ability to intercept/modify all communications from the client/server connection.

In [ ]:
class Interceptor:
    def __init__(self):
        ...
        
    def interceptMessage(self, message):
        """
        The message will be of the form 
        { 
            sender: 'client/server',
            receiver: 'client/server',
            type: 'certificate/ECDH_Parameters/ECDH_Message/request',
            data: ...
        }
        """
        return message['data'] # Default behavior: just pass the message along
        
    def getPassword(self):
        return None

Here's an example to test with:

In [ ]:
random.seed(0)

# New client
client = Client()

# Client initiates server connection
server = Server()

server.LOGINS = {}

# Initiate Interceptor
intercept = Interceptor()

message = intercept.interceptMessage({'sender': 'client', 'receiver': 'server',
                                              'type': 'CLIENT_RSA', 'data': client.RSA_PUBLIC_KEY})
if message:
    server.acceptConnection(message)


server_PK, signature = server.publishRSAPublicKey()

message = intercept.interceptMessage({'sender': 'server', 'receiver': 'client',
                                              'type': 'certificate', 'data': (server_PK, signature)})
assert client.verifyServerCertificate(*message)

# Once verified, ask for ECDH parameters
ellipticCurve, ellipticCurveModulus, generatorPoint, generatorPointOrder, signature = server.generateECDHParameters()

message = intercept.interceptMessage({'sender': 'server', 'receiver': 'client',
                                              'type': 'ECDH_Parameters', 'data': (ellipticCurve, ellipticCurveModulus,
                                                                                generatorPoint, generatorPointOrder, signature)})

assert client.verifyECDHParameters(*message)

# Once ECDH parameters are verified, generate our secret + public value
server_ECDH = server.generateSignedECDHMessage()

messageOne = intercept.interceptMessage({'sender': 'server', 'receiver': 'client',
                                              'type': 'ECDH_Message', 'data': server_ECDH})
client_ECDH = client.generateSignedECDHMessage()

messageTwo = intercept.interceptMessage({'sender': 'client', 'receiver': 'server',
                                              'type': 'ECDH_Message', 'data': client_ECDH})
if messageOne:
    client.acceptECDHMessage(*messageOne)
if messageTwo:
    server.acceptECDHMessage(*messageTwo)


signupRequest = {'data': '{"iv": "Kmy+SVbVlbVcPXYGW4koSA==", "ciphertext": "2ir3du/Q4RV9Iatux4U4UdyA/+LXs2xVLnZ2zBneyqPOfEID2QegaSLCd4hqJCAekeHwpzhLWitHL+lbf301smd7FfrXQlLbsJATDiSAvxQ="}',
 'hmac': '1f41c33271f8418d9147e22bc16f5dfcb921b3f30d544de2d22cbb5d300638d8'}

message = intercept.interceptMessage({'sender': 'client', 'receiver': 'server',
                                              'type': 'request', 'data': signupRequest})

In [ ]:
intercept.getPassword()

### Submission

Once you've recovered the password, enter it into the Gradescope page [here](https://www.gradescope.com/courses/406561/assignments/2461065) and copy-paste **all** your code into the same assignment.